Ok, now let's make a model. The paper I'm modelling this project on uses a modified version of Structural Deep Neighbor Embedding (SDNE): https://www.kdd.org/kdd2016/papers/files/rfp0191-wangAemb.pdf. There's a very nice implementation of SDNE in keras (along with some other embeddings) here: https://github.com/palash1992/GEM. Instead of creating SDNE from scratch, I'm just going to modify the SDNE model in the GEM package.

The SDNE model in the GEM package takes one large graph and trains on batches of its edges. There are a few ways we could extend this to our problem- we could pad each team's graph with isolated nodes and feed each team's graph in one at a time, or we could create one big graph formed of several disjoint connected-component graphs consisting of each team's roster. The latter makes more sense here- we want to learn the embedding of each individual player node in the training set and how the properties of each node can be used to predict unobserved edges, not necessarily a set of graph properties that can be generalized across teams (which is a harder problem). In other words, for now, I'm interpolating, not extrapolating. For generalizable properties, we'd likely need to look at stuff like GCNs or GAT models- things that learn bigger structural properties.

There is one small assumption that goes into this- that two blockers with the same name are the same player. Derby names are punny, so it's not rare that a good player name like 'Smackpropagation' will show up on a few different teams. However, the nature of our pruning algorithm tends to only give us data for graphs for decently established teams, and this name-similarity problem tends to only show up with more local teams (who are less likely to have enough stats on the website).

The Teammate algorithm randomly removes 20% of the edges from the network, sets this aside as a test network and the non-removed 80% as a train network. We need to do this too.

Because we are only learning observed edges, not null ones, and the random train-test subsampling procedure described in the Team Composition paper can create disconnected graphs that are then fed into the autoencoder, it doesn't seem like strongly-connectedness is necessarily important for the Teammate algorithm to work (just that it's considered good form). Spectral-decomposition based algorithms do require connectivity, but I don't believe this one does.

The Teammate paper uses random-walk sampling on the test set. Because our graph is made of several disjoint components rather than one large LCC, I'm also not entirely convinced this step will do anything for us, so I'll leave it out for now.

-Train on the train network. The loss function should be the same as SDNE i.e. (L = $\sum_{i=1}^{n} |(\hat{x}_{i}-x_{i})  \odot [b_{(i,j)}]_{j=1}^{n}|^{2}$) but with the follwing change:

Base SDNE: $b_{(i,j)} = 1$ if $(i,j)= 0, b_{(i,j)} = \beta > 1$ otherwise.

Teammate Encoder SDNE: $b_{(i,j)} = 0$ if $(i,j)= 0, b_{(i,j)} = 1$ otherwise.

In standard SDNE, the loss function penalizes zero edges. However, we don't just want to penalize them- we want to completely ignore them. We're not trying to predict *if* players have played together, just how well they do play together when they do. The base SDNE also has a parameter $\alpha$ that controls the first-order loss via the method of Laplacian Eigenmaps, but if we set this to zero we will reconstruct the 'simple' loss function given in the Teammate paper.

These modified embedding methods are implemented in 'teammate.py' and 'teammate_utils.py',

We'll also need to implement the MSE and MANE methods of measuring graph reconstruction ability. I've added these metrics to the 'metrics.py' file.

In [ ]:
'''
Run the graph embedding methods on Karate graph and evaluate them on 
graph reconstruction and visualization. Please copy the 
gem/data/karate.edgelist to the working directory
'''
import matplotlib.pyplot as plt
from time import time

from gem.utils      import graph_util, plot_util
from gem.evaluation import visualize_embedding as viz
from gem.evaluation import evaluate_graph_reconstruction as gr

from gem.embedding.gf       import GraphFactorization
from gem.embedding.hope     import HOPE
from gem.embedding.lap      import LaplacianEigenmaps
from gem.embedding.lle      import LocallyLinearEmbedding
from gem.embedding.node2vec import node2vec
import networkx as nx
from gem.embedding.teammate     import Teammate
from argparse import ArgumentParser


if __name__ == '__main__':
    ''' Sample usage
    python run_karate.py -node2vec 1
    '''
    parser = ArgumentParser(description='Graph Embedding Experiments on Roller Derby graphs')
    parser.add_argument('-node2vec', '--node2vec',
                        help='whether to run node2vec (default: False)')
    args = vars(parser.parse_args())
    try:
        run_n2v = bool(int(args["node2vec"]))
    except:
        run_n2v = False

    # File that contains the edges. Format: source target
    # Optionally, you can add weights as third column: source target weight
    edge_tot = '../../Data/AllTeamsFullLTGraph.edgelist'
    edge_train = '../../Data/AllTeamsLTGraphTrainStandardized.edgelist'
    edge_test = '../../Data/AllTeamsLTGraphTestStandardized.edgelist'
    edge_val = '../../Data/AllTeamsLTGraphValStandardized.edgelist'
    # Specify whether the edges are directed
    isDirected = True

    # Load graph. Have to prune manually to keep number of nodes fixed
    G = nx.read_weighted_edgelist(edge_tot, nodetype=int)
    G_test_dummy = nx.read_weighted_edgelist(edge_test, nodetype=int)
    G_train_dummy = nx.read_weighted_edgelist(edge_train, nodetype=int)
    G_val_dummy = nx.read_weighted_edgelist(edge_val, nodetype=int)

    G = G.to_directed()
   G = G.to_directed()

    G_train = G.copy()
    G_val = G.copy()
    G_test = G.copy()

    for edge in G.edges():
        if edge not in G_train_dummy.edges(): G_train.remove_edge(*edge)
        if edge not in G_test_dummy.edges(): G_test.remove_edge(*edge)
        if edge not in G_val_dummy.edges(): G_val.remove_edge(*edge)

    print(len(G_train))
    print(len(G_test))
    print(len(G_val))

    print(G.number_of_edges())
    print(G_train.number_of_edges())
    print(G_val.number_of_edges())
    print(G_test.number_of_edges())
    print(G_train_dummy.nodes)

    models = []
    # Load the models you want to run
    # models.append(GraphFactorization(d=2, max_iter=50000, eta=1 * 10**-4, regu=1.0, data_set='derby'))
    #models.append(HOPE(d=4, beta=0.01))
    #models.append(LaplacianEigenmaps(d=2))
    #models.append(LocallyLinearEmbedding(d=2))
    if run_n2v:
        models.append(
            node2vec(d=2, max_iter=1, walk_len=80, num_walks=10, con_size=10, ret_p=1, inout_p=1)
        )
    #alpha = 0 to have "traditional" second order loss
    models.append(Teammate(d=2, beta=5, alpha=0, nu1=1e-6, nu2=1e-6, K=3,n_units=[500, 300], rho=0.3, n_iter=50, xeta=0.001, n_batch=7,
                    modelfile=['enc_model.json', 'dec_model.json'],
                    weightfile=['enc_weights.hdf5', 'dec_weights.hdf5']))

    # For each model, learn the embedding and evaluate on graph reconstruction and visualization
    for embedding in models:
        print ('Num nodes: %d, num edges: %d' % (G.number_of_nodes(), G.number_of_edges()))
        t1 = time()
        # Learn embedding - accepts a networkx graph or file with edge list
        Y, t = embedding.learn_embedding(graph=G_train, edge_f=None, is_weighted=True, no_python=True)
        print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))
        print(Y)
 
        # Evaluate on graph reconstruction:train
        MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G_train, embedding, Y, None, is_weighted=True, is_undirected=False)
        print(("\tMAP: {} \t precision curve: {}\n\n\n\n"+'-'*100).format(MAP,prec_curv[:5]))
        viz.plot_embedding2D(embedding.get_embedding(), di_graph=G_train, node_colors=None)
        plt.show()
        plt.clf()

        # Evaluate on graph reconstruction:val
        MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G_val, embedding, Y, None, is_weighted=True, is_undirected=False)
        print(("\tMAP: {} \t precision curve: {}\n\n\n\n"+'-'*100).format(MAP,prec_curv[:5]))
        viz.plot_embedding2D(embedding.get_embedding(), di_graph=G_val, node_colors=None)
        plt.show()
        plt.clf()

"""
        # Evaluate on graph reconstruction:val
        MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G_test, embedding, Y, None, is_weighted=True, is_undirected=False)
        print(("\tMAP: {} \t precision curve: {}\n\n\n\n"+'-'*100).format(MAP,prec_curv[:5]))
        viz.plot_embedding2D(embedding.get_embedding(), di_graph=G_test, node_colors=None)
        plt.show()
        plt.clf()

"""
